In [ ]:
import pandas as pd
from sklearn.metrics import f1_score




def train(train_df,test_df):
    train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
    train_df[['text','label_ft']].to_csv('train.csv', index=None, header=None, sep='\t')

    import fasttext
    model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2,
                                  verbose=2, minCount=1, epoch=50, loss="hs")


    val_pred = [model.predict(x)[0][0].split('__')[-1] for x in test_df['text']]
    print(f1_score(test_df['label'].values.astype(str), val_pred, average='macro'))



In [ ]:
# 转换为FastText需要的格式
from sklearn.model_selection import KFold
train_df = pd.read_csv('/home/hy/Documents/tianchi/train_set.csv', sep='\t', nrows=20000)
KF = KFold(n_splits=10,shuffle=True)

for train_index,test_index in KF.split(train_df):
    train(train_df.iloc[train_index].copy(deep=True),train_df.iloc[test_index].copy(deep=True))



In [ ]:
train_df = pd.read_csv('/home/hy/Documents/tianchi/train_set.csv', sep='\t', nrows=20000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].to_csv('train.csv', index=None, header=None, sep='\t')
def BayesCV(lr,word_ngrams,verbose,minCount,epoch):
    import fasttext
    model = fasttext.train_supervised('train.csv', lr=lr, wordNgrams=word_ngrams,
                                  verbose=verbose, minCount=minCount, epoch=epoch, loss="hs")
    val_pred = [model.predict(x)[0][0].split('__')[-1] for x in test_df['text']]
    return f1_score(test_df['label'].values.astype(str), val_pred, average='macro')
    
gbdt_op = BayesianOptimization(
        BayesCV,
        {
        'lr': (0, 1),
        'word_ngrams': (1, 2),
        'verbose': (0, 2),
        'minCount': (1, 2),
        'epoch':(30,55)
        }
    )

gbdt_op.maximize()